# Enums

* Enums are a powerful feature in Rust that allow you to define a type by enumerating its possible variants.

* Enums can have data associated with each variant, making them more flexible than simple enums in other languages.

* Pattern matching with enums allows you to concisely handle different cases, making your code more expressive and safe.

## C-Style Enums

* The simplest enums are just a list of variants without any associated data.
* They are also called "fieldless" or "unit" variants.
* They work similar to enums in C/C++

In [2]:
#[derive(Debug)]
enum Compass {
    North,
    East,
    South,
    West
}

let direction = Compass::North;

println!("direction = {:?}", direction);

direction = North


* In memory, values of C-style enums are stored as integers. 

In [3]:
use std::mem::size_of;

assert!(size_of::<Compass>() == 1);

### Importing constructors

In [4]:
use self::Compass::*; // self - when enum type is defined in current module; otherwise name of a module

let direction = North;

### Using Integral Values

In [5]:
#[derive(Debug)]
enum HttpStatus {
    Ok = 200,
    NotModified = 304,
    NotFound = 404
}

assert!(size_of::<HttpStatus>() == 2);

### Casting to Integer

* "Fieldless" or "unit" variants can be converted to integral value using the `as` keyword.

In [6]:
let ok_code = HttpStatus::Ok as i32;
assert!(ok_code == 200);

assert!(HttpStatus::NotFound as i32 == 404);

### Casting from Integer

* Converting from integral value requires pattern matching.

In [7]:
fn http_status_from_u32(status: u32) -> Option<HttpStatus> {
    match status {
        200 => Some(HttpStatus::Ok),
        304 => Some(HttpStatus::NotModified),
        404 => Some(HttpStatus::NotFound),
        _ => None
    }
}

In [8]:
http_status_from_u32(404)

Some(NotFound)

In [9]:
http_status_from_u32(1024)

None

### Deriving From Common Traits

* As with structs, the compiler will implement features like the `==` operator for you, but you have to ask.

In [10]:
#[derive(Copy, Clone, Debug, PartialEq)]
enum TimeUnit {
    Seconds, Minutes, Hours, Days, Months, Years
}

In [11]:
assert!(TimeUnit::Seconds == TimeUnit::Seconds);

### Implementing Methods for Enums

* We can add methods to enums using `impl` blocks

In [12]:
impl TimeUnit {
    /// Return the plural noun for this time unit.
    fn plural(self) -> &'static str {
        match self {
            TimeUnit::Seconds => "seconds",
            TimeUnit::Minutes => "minutes",
            TimeUnit::Hours => "hours",
            TimeUnit::Days => "days",
            TimeUnit::Months => "months",
            TimeUnit::Years => "years"
        }
    }

    /// Return the singular noun for this time unit.
    fn singular(self) -> &'static str {
        self.plural().trim_right_matches('s')
    }
}

In [13]:
let time_unit = TimeUnit::Days;

time_unit.plural()

"days"

## Enums With Data

* The power of Rust enums lies in their ability to store additional data
* This allows to create complex data types that can hold different kinds of information

In [14]:
#[derive(Clone, Debug, PartialEq)]
enum Message {
    Quit, // no data associated
    Move{ x: i32, y: i32 }, // named fields - like a struct
    Write(String), // single String
    ChangeColor(i32, i32, i32) // tuple struct of three values
}

In [15]:
let mut msg = Message::Quit;
println!("msg = {:?}", msg);

msg = Message::Move{ x: 100, y: 200};
println!("msg = {:?}", msg);

msg = Message::Write(String::from("TEXT"));
println!("msg = {:?}", msg);

msg = Message::ChangeColor(255, 0, 255);
println!("msg = {:?}", msg);

msg = Quit
msg = Move { x: 100, y: 200 }
msg = Write("TEXT")
msg = ChangeColor(255, 0, 255)


In [16]:
impl Message {
    fn call(&self) {
        match *self {
            Message::Quit => println!("Quit"),
            Message::Move{ x, y } => println!("Move to x={}, y={}", x, y),
            Message::Write(ref text) => println!("Write '{}'", text),
            Message::ChangeColor(r, g, b) => println!("Change color to r={}, g={}, b={}", r, g, b)
        }
    }
}

In [17]:
msg.call();

Change color to r=255, g=0, b=255


## Generic Enums

* Enums can be parametrized with type parameters to create generic enums.

### `Option<T>`

In [18]:
mod explain {
    enum Option<T> {
        None,
        Some(T)
    }
}

### `Result<T, E>`

In [19]:
mod explain{
    enum Result<T, E> { 
        Ok(T), 
        Err(E) 
    }
} 

### Real-World Example - BinaryTree

In [20]:
// An ordered collection of `T`s.
enum BinaryTree<T> {
    Empty,
    NonEmpty(Box<TreeNode<T>>)
}

// A part of a BinaryTree.
struct TreeNode<T> {
    element: T,
    left: BinaryTree<T>,
    right: BinaryTree<T>
}

In [21]:
use self::BinaryTree::*;

let jupyter_tree = NonEmpty(Box::new(TreeNode {
    element: "Jupyter",
    left: Empty,
    right: Empty
}));

let mercury_tree = NonEmpty(Box::new(TreeNode {
    element: "Mercury",
    left: Empty,
    right: Empty
}));

let venus_tree = NonEmpty(Box::new(TreeNode {
    element: "Venus",
    left: Empty,
    right: Empty
}));

let mars_tree = NonEmpty(Box::new(TreeNode {
    element: "Mars",
    left: jupyter_tree,
    right: mercury_tree
}));

let uranus_tree = NonEmpty(Box::new(TreeNode {
    element: "Uranus",
    left: Empty,
    right: venus_tree
}));

// the root
let saturn_tree = NonEmpty(Box::new(TreeNode {
    element: "Saturn",
    left: mars_tree,
    right: uranus_tree
}));

## Pattern Matching

* Pattern matching allows to compare a value against a series of patterns and execute code based on which pattern matches.
* Patterns can include literals, variable names, wildcards, and more complex structures.
* Patterns are commonly used with the `match` expression in Rust.
* Match arms consist of a pattern and the code to execute if the pattern matches.
* `match` expression is **infallible** and must cover all possible cases.

In [22]:
enum RoughTime {
    InThePast(TimeUnit, u32),
    JustNow,
    InTheFuture(TimeUnit, u32)
}

In [23]:
fn rough_time_to_english(rt: RoughTime) -> String {
    match rt {
        RoughTime::InThePast(units, 1) => format!("{} ago", units.singular()),
        RoughTime::InThePast(units, count) => format!("{} {} ago", count, units.plural()),
        RoughTime::JustNow => format!("just now"),
        RoughTime::InTheFuture(units, 1) => format!("{} from now", units.singular()),  
        RoughTime::InTheFuture(units, count) => format!("{} {} from now", count, units.plural())  
    }
}

In [24]:
rough_time_to_english(RoughTime::InThePast(TimeUnit::Seconds, 1))

"second ago"

In [25]:
rough_time_to_english(RoughTime::InThePast(TimeUnit::Seconds, 42))

"42 seconds ago"

In [26]:
rough_time_to_english(RoughTime::JustNow)

"just now"

In [27]:
rough_time_to_english(RoughTime::InTheFuture(TimeUnit::Seconds, 42))

"42 seconds from now"

|      Pattern type       |        Example         |                                   Notes                                    |
| ----------------------- | ---------------------- | -------------------------------------------------------------------------- |
| Literal                 | `100`                  | Matches an exact value; the name of a const is also allowed                |
|                         | `"name"`               |                                                                            |
| Range                   | `0 ... 100`            | Matches any value in range, including the end value                        |
|                         | `'a' ... 'k'`          |                                                                            |
| Wildcard                | `_`                    | Matches any value and ignores it                                           |
| Variable                | `name`                 | Like _ but moves or copies the value into a new local variable             |
|                         | `mut count`            |                                                                            |
| ref variable            | `ref field`            | Borrows a reference to the matched value instead of moving or copying it   |
|                         | `ref mut field`        |                                                                            |
| Binding with subpattern | `val @ 0 ... 99`       | Matches the pattern to the right of @, using the variable name to the left |
|                         | `ref circle @`         |                                                                            |
|                         | `Shape::Circle { .. }` |                                                                            |
| Enum pattern	          | `Some(value)`          |                                                                            |
|                         | `None`                 |                                                                            |
|                         | `Message::Ok`          |                                                                            |
| Tuple pattern	          | `(key, value)`         |                                                                            |
|                         | `(r, g, b)`            |                                                                            |
| Struct pattern	      | `Color(r, g, b)`       |                                                                            |
|                         | `Point { x, y }`       |                                                                            |
|                         | `Card { suit: Clubs, rank: n }`  |                                                                  |
|                         | `Account { id, name, .. }`       |                                                                  | 
| Reference	              | `&value`               | Matches only reference values                                              |
|                         | `&(k, v)`	           |                                                                            |
| Multiple patterns	      | `'a' \| 'A'`           | In match only (not valid in let, etc.)                                     |
| Guard expression	      | `x if x * x <= r2`     | In match only (not valid in let, etc.)                                     |

### Literals, Variables and Wildcards in Patterns

In [28]:
let no_of_clicks = 4;

match no_of_clicks {
    0 => format!("No one clicked..."),
    1 => format!("Just one click..."),
    n => format!("{} clicks", n)
}

"4 clicks"

* Rust requires every match expression to handle all possible values, a wildcard is often required at the end
* This ensures that the match is exhaustive and prevents runtime errors.

In [29]:
let http_code = HttpStatus::NotFound;

match http_code {
    HttpStatus::Ok => "Ok",
    _ => "Not Ok"
}

"Not Ok"

### Tuple Patterns

In [30]:
fn describe_point(x: i32, y:i32) -> &'static str {
    use std::cmp::Ordering::*;

    match (x.cmp(&0), y.cmp(&0)) {
        (Equal, Equal) => "at the origin",
        (_, Equal) => "on the x axis",
        (Equal, _) => "on the y axis",
        (Greater, Greater) => "in the first quadrant",
        (Less, Greater) => "in the second quadrant",
        _ => "somewhere else"
    }
}

In [31]:
describe_point(0, 42)

"on the y axis"

### Struct Patterns

In [32]:
#[derive(Debug, PartialEq)]
struct Point {
    x: i32, 
    y: i32
}

let location = Point{x:0, y:100};

match location {
    Point{x: 0, y: height} => format!("straight up {} meters", height),
    Point{x: x, y: y } => format!("at ({},{})", x, y)
}

"straight up 100 meters"

* Patterns like `Point { x: x, y: y }` are common when matching structs, and the redundant names are visual clutter, so Rust has a shorthand for this: `Point {x, y}` - this pattern still stores a point’s `x` field in a new local `x` and its `y` field in a new local `y`.

In [33]:
match location {
    Point{x: 0, y} => format!("straight up {} meters", y),
    Point{x, y} => format!("at ({},{})", x, y)
}

"straight up 100 meters"

* Matching larger structs

In [34]:
#[derive(Debug)]
struct Person {
    id: u64,
    first_name: String,
    last_name: String,
    age: u8,
    salary: f32,
    job: String
}

In [35]:
let person = Person{id: 665, first_name: "Jan".to_string(), last_name: "Kowalski".to_string(), age: 32, salary:9000.99, job: "Rust Developer".to_string() };

match person {
    Person{id: _, first_name, last_name, age: _, salary: _, job: _} => format!("{} {}", first_name, last_name)
}

"Jan Kowalski"

* We can skip uninteresting fields with `..`

In [36]:
let person = Person{id: 665, first_name: "Jan".to_string(), last_name: "Kowalski".to_string(), age: 32, salary:9000.99, job: "Rust Developer".to_string() };

match person {
    Person{first_name, last_name, ..} => format!("{} {}", first_name, last_name)
}

"Jan Kowalski"

### Reference Patterns

#### `ref` Patterns

* `ref` patterns borrow parts of a matched value

In [37]:
fn print_names(person: &Person) {
    print!("{} {}", person.first_name, person.last_name);    
}

In [38]:
let person = Person{ id: 665, first_name: "Jan".to_string(), last_name: "Kowalski".to_string(), age: 32, salary:9000.99, job: "Rust Developer".to_string() };

match person {
    Person{ job, .. } => { print_names(&person); println!("Works as {}", job) }
}

Error: use of deprecated method `core::str::<impl str>::trim_right_matches`: superseded by `trim_end_matches`

Error: borrow of partially moved value: `person`

In [39]:
let person = Person{ id: 665, first_name: "Jan".to_string(), last_name: "Kowalski".to_string(), age: 32, salary:9000.99, job: "Rust Developer".to_string() };

match person {
    Person{ ref job, .. } => { print_names(&person); println!(" Works as {}", job) }
}

Jan Kowalski Works as Rust Developer


()

In [40]:
fn to_upper(text: &mut String) {
    *text = text.to_uppercase();
}

let mut person = Person{ id: 665, first_name: "Jan".to_string(), last_name: "Kowalski".to_string(), age: 32, salary:9000.99, job: "Rust Developer".to_string() };

match person {
    Person{ref mut first_name, ref mut last_name, ..} => { to_upper(first_name); to_upper(last_name) }
}

person

Person { id: 665, first_name: "JAN", last_name: "KOWALSKI", age: 32, salary: 9000.99, job: "Rust Developer" }

#### & Pattern

* A pattern starting with `&` matches a reference.

In [41]:
#[derive(Debug, PartialEq)]
struct ColorCircle { pos: Point, r: u32, color: (u8, u8, u8) }

impl ColorCircle {
    fn center(&mut self) -> &mut Point {
        return &mut self.pos;
    }
}

let mut c = ColorCircle{pos: Point{x: 100, y: 200}, r: 50, color: (255, 0, 0)};

match c.center() {
    &mut Point{x, y} => { format!("x={}, y={}", x, y) },
}

"x=100, y=200"

In [42]:
match c.center() {
    &mut Point{ref mut x, ..} => { *x = 10; },
}

c.center()

Point { x: 10, y: 200 }

In [43]:
#[derive(Debug)]
struct Item(String);

let items = [Item("one".to_string()), Item("two".to_string()), Item("three".to_string())];

{
    let mut iter = items.iter().peekable();

    match iter.peek() {
        Some(&item) => println!("item peeked: {:?}", item),
        None => println!("no item")
    }
}

item peeked: Item("one")


()

### Matching Multiple Possibilities

In [44]:
let text = String::from("\nBC");

{
    let mut chars = text.chars().peekable();

    let at_end = match chars.peek() {
        Some(&'\r') | Some(&'\n') | None => true,
        _ => false
    };

    at_end
}

true

In [45]:
let text = String::from("aB12 -&^%$# 34cd\r\n");

for c in text.chars() {
    match c {
        'a'..='z' => println!("lowercase letter: {}", c),
        'A'..='Z' => println!("uppercase letter: {}", c),
        '0'..='9' => println!("digit: {}", c),
        ' ' | '\t' => println!("whitespace"),
        '&' | '^' | '%' | '$' | '#' => println!("special character: {}", c),
        '\n' => println!("newline"),
        _ => println!("other character")
    }
}

lowercase letter: a
uppercase letter: B
digit: 1
digit: 2
whitespace
other character
special character: &
special character: ^
special character: %
special character: $
special character: #
whitespace
digit: 3
digit: 4
lowercase letter: c
lowercase letter: d
other character
newline


()

### Pattern Guards

* Pattern guards allow to add additional conditional checks to patterns in match arms.

In [46]:
struct Point3D {
    x: f64,
    y: f64,
    z: f64
}

impl Point3D {
    fn new(x: f64, y: f64, z: f64) -> Point3D {
        Point3D{x: x, y: y, z: z}
    }

    fn distance_to_origin(&self) -> f64 {
        (self.x.powf(2.0) + self.y.powf(2.0) + self.z.powf(2.0)).sqrt()
    }
}

let point = Point3D::new(0.3, 0.4, 0.5);

match point {
    Point3D{x, y, z} if point.distance_to_origin() <= 1.0 => format!("Inside the sphere: x={}, y={}, z={}", x, y, z),
    Point3D{x, y, z} => format!("Outside the sphere: x={}, y={}, z={}", x, y, z)
}

"Inside the sphere: x=0.3, y=0.4, z=0.5"

### `@` Patterns

* Finally, `x @` pattern matches exactly like the given pattern, but on success, instead of creating variables for parts of the matched value, it creates a single variable `x` and moves or copies the whole value into it

In [47]:
let number = Some(5);

match number {
    Some(n @ 1..=5) => println!("Matched: {}", n),
    Some(n) => println!("Other: {}", n),
    None => println!("No value"),
}

Matched: 5


()

In [48]:
let point = Point { x: 5, y: 10 };

match point {
    Point { x: n @ 1..=5, y } => println!("x is in range 1 to 5: {}; y is {}", n, y),
    Point { x, y } => println!("x is {}; y is {}", x, y),
}

x is in range 1 to 5: 5; y is 10


()

In [49]:
#[derive(Debug, PartialEq)]
struct Circle{ center: Point, radius: u32 }

#[derive(Debug, PartialEq)]
struct Rectangle{ top_left: Point, width: u32, height: u32 }

#[derive(Debug, PartialEq)]
enum Shape {
    Circle(Circle),
    Rectangle(Rectangle)
}

fn inflate(shape: &mut Shape, factor: u32) {
    match shape {
        Shape::Circle(circle @ Circle{..}) => {
            circle.radius *= factor;
        },
        Shape::Rectangle(rect @ Rectangle{..}) => {
            rect.width *= factor; 
            rect.height *= factor; 
        }
    }
}

let mut s = Shape::Circle(Circle{center: Point{x: 100, y: 200}, radius: 50});

inflate(&mut s, 2);

assert!(s == Shape::Circle(Circle{center: Point{x: 100, y: 200}, radius: 100}));

### Where Patterns Can Be Applied

#### Irrefutable patterns

* Patterns that always match are special in Rust. 
* They’re called **irrefutable patterns**
* They’re the only patterns allowed places shown below

#### Unpack a struct into three new local variables

In [50]:
let point = Point3D::new(1.0, 2.0, 3.0);

let Point3D{x, y, ..} = point; // unpacking

println!("x={}, y={}", x, y)

x=1, y=2


()

#### Unpack a function argument that's a tuple

In [51]:
fn distance_to((x, y): (f64, f64)) -> f64 { 
    (x.powf(2.0) + y.powf(2.0)).sqrt() 
}

distance_to((3.0, 4.0))

5.0

#### Iterate over keys and values of a HashMap

In [52]:
use std::collections::HashMap;

let mut dictionary = HashMap::new();
dictionary.insert("one", 1);
dictionary.insert("two", 2);
dictionary.insert("three", 3);

for (key, value) in &dictionary {
    println!("{}: {}", key, value);
}

three: 3
one: 1
two: 2


()

#### Refutable Patterns

* A **refutable pattern** is one that might not match, like `Ok(x)`, which doesn’t match an error result, or `'0' ... '9'`, which doesn’t match the character `'Q'`

* Refutable patterns are also allowed in `if let` and `while let` expressions, which can be used to...

In [53]:
fn date_of_birth() -> RoughTime {
    RoughTime::InThePast(TimeUnit::Years, 32)
}

if let RoughTime::InThePast(units, count) = date_of_birth() {
    println!("{} years ago", count);
}

32 years ago


()

### Real-World Example - BinaryTree - 2

In [2]:
// An ordered collection of `T`s.
enum BinaryTree<T> {
    Empty,
    NonEmpty(Box<TreeNode<T>>)
}

// A part of a BinaryTree.
struct TreeNode<T> {
    element: T,
    left: BinaryTree<T>,
    right: BinaryTree<T>
}

In [3]:
impl<T: Ord> BinaryTree<T> {
    fn add(&mut self, value: T) {
        match *self {
            BinaryTree::Empty => {
                *self = BinaryTree::NonEmpty(Box::new(TreeNode {
                    element: value,
                    left: BinaryTree::Empty,
                    right: BinaryTree::Empty
                }));
            },
            BinaryTree::NonEmpty(ref mut node) => {
                if value <= node.element {
                    node.left.add(value);
                } else {
                    node.right.add(value);
                }
            }
        }
    }

    fn preorder<'a>(&'a self, values: &mut Vec<&'a T>) {
        match *self {
            BinaryTree::Empty => {},
            BinaryTree::NonEmpty(ref node) => {
                values.push(&node.element);
                node.left.preorder(values);
                node.right.preorder(values);
            }
        }
    }

    fn postorder<'a>(&'a self, values: &mut Vec<&'a T>) {
        match *self {
            BinaryTree::Empty => {},
            BinaryTree::NonEmpty(ref node) => {
                node.left.postorder(values);
                node.right.postorder(values);
                values.push(&node.element);
            }
        }
    }
}

In [4]:
let mut tree = BinaryTree::Empty;

{
    tree.add("Givry".to_string());
    tree.add("Mercurey".to_string());
    tree.add("Rully".to_string());
    tree.add("Beaune".to_string());
    tree.add("Pommard".to_string());

    let mut values: Vec<&String> = Vec::new();
    tree.preorder(&mut values);

    values
}

["Givry", "Beaune", "Mercurey", "Rully", "Pommard"]

In [6]:
let mut another_tree = BinaryTree::Empty;

{
    another_tree.add("Chardonnay");
    another_tree.add("Tannat");
    another_tree.add("Cabernet Sauvignon");
    another_tree.add("Merlot");
    another_tree.add("Pinot Noir");
    another_tree.add("Solaris");

    let mut values = Vec::new();
    another_tree.postorder(&mut values);
    values
}

["Cabernet Sauvignon", "Solaris", "Pinot Noir", "Merlot", "Tannat", "Chardonnay"]